## Case summary for the 3d project

### Load modules

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from scipy.interpolate import interp1d
import json

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
TwoDSubduction_DIR = os.environ['TwoDSubduction_DIR']
ThDSubduction_DIR = os.environ['ThDSubduction_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
from shilofue.ThDSubduction0.Group import CASE_SUMMARY
from shilofue.ThDSubduction0.PlotCase import ExportStrenghProfile
from shilofue.TwoDSubduction0.Group import CASE_SUMMARY as CASE_SUMMARY_2D
import shilofue.ThDSubduction0.VtkPp as ThDVtkPp
from shilofue.TwoDSubduction0.VtkPp import SLABPLOT as SLABPLOT2D
from shilofue.TwoDSubduction0.VtkPp import GetSlabDipAt660
from shilofue.Group import ReadBasicInfoGroup
from shilofue.ThDSubduction0.PlotVisit import VISIT_OPTIONS
from shilofue.TwoDSubduction0.PlotVisit import VISIT_OPTIONS as VISIT_OPTIONS_2D
from shilofue.Analytics import CAP13

In [ ]:
group_dir = os.path.join(ThDSubduction_DIR, "chunk_test")
assert(os.path.isdir(group_dir))
group_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_13")
assert(os.path.isdir(group_dir_2d))

### 3d case summary

In [ ]:
all_actions=['geometry', 't660', 't800', 't1000', 'dip660', 'ages', 'strength', 'include_Ps']

# for the project, no update, hand correct
write_for_paper = False; update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary.tex')
# for one group, update from group
# write_for_paper = False; update_from_group = True; o_path = os.path.join(group_dir, 'case_summary.txt'); o_path_csv = os.path.join(group_dir, 'case_summary.csv'); o_path_tex = os.path.join(group_dir, 'case_summary.tex')

# for export to a latex table for the paper
# write_for_paper = True; update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary_3d_paper.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary_3d_paper.tex')


img_dir = os.path.join(group_dir, 'img')
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

Case_Summary = CASE_SUMMARY(VISIT_OPTIONS=VISIT_OPTIONS)

# import old result if it exists 
if os.path.isfile(o_path_csv):
  Case_Summary.import_file(o_path_csv)
  Case_Summary.Update(actions=all_actions)

# import the new directory
if update_from_group:
  Case_Summary.import_directory(group_dir, actions=all_actions)

# output file output
if write_for_paper: 
  Case_Summary.write_file_for_paper(o_path_tex)
else:
  Case_Summary.write_file(o_path)
  Case_Summary.write_file(o_path_csv)
  Case_Summary.write_file(o_path_tex)

#### Plot Ribe 10, figure 11

In [ ]:
sp_ages = np.array(Case_Summary.sp_ages)
sp_thickness = np.minimum(11.2*sp_ages**0.5, 100.0)
print("sp_thickness: ", sp_thickness)

In [ ]:
sp_ages = [80.0, 80.0, 80.0, 80.0, 40.0, 40.0, 40.0, 40.0] # Ma
slab_strs= [1e6, 1000.0, 300.0, 100.0, 1e6, 1000.0, 300.0, 100.0] # MPa
include_peierls_arr = [False, True, True, True, False, True, True, True]
# dip660s = [82.8608]
case_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction/EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"
um_thickness = 660e3
strain_rate = 1e-15
eta_m_ref = 1e20

case_dir_for_plot = "/mnt/lochy/ASPECT_DATA/ThDSubduction/EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"
sp_age_for_plot = 80.0
slab_str_for_plot= 1000.0
o_path = os.path.join(ThDSubduction_DIR, "slab_strength_profile_SA_%.1f_s%.1f.pdf"  % (sp_age_for_plot, slab_str_for_plot))
ExportStrenghProfile(case_dir_for_plot, sp_age_for_plot, slab_str_for_plot*1e6, strain_rate, o_path)
print("saved figure %s" % o_path)

eta_aver_list = []
eta_max_list = []
sp_thickness_list = []
for i in range(len(sp_ages)):
    slab_str = slab_strs[i] * 1e6
    sp_age = sp_ages[i]
    sp_thickness = np.minimum(11.2*sp_age**0.5, 100.0) * 1e3 # m
    include_peierls = include_peierls_arr[i]

    depths, taus, etas = ExportStrenghProfile(case_dir, sp_age, slab_str, strain_rate, include_peierls=include_peierls)
    mask = (depths < sp_thickness)

    integral = np.trapz(np.log(etas[mask]), depths[mask])
    eta_aver = np.exp(integral / sp_thickness)
    eta_max = np.max(etas[mask])
    print("eta_aver: ", eta_aver)
    print("eta_max: ", eta_max)
    
    sp_thickness_list.append(sp_thickness)
    eta_aver_list.append(eta_aver)
    eta_max_list.append(eta_max)

sp_thickness_array = np.array(sp_thickness_list)
eta_aver_array = np.array(eta_aver_list)
eta_max_array = np.array(eta_max_list)
# dip660_array = np.array(dip660s)

gamma_array = eta_max_array / eta_m_ref
D_to_h_array = um_thickness / sp_thickness_array

fig, ax = plt.subplots(figsize=(5, 5))
ax.semilogx(gamma_array, D_to_h_array, '*')
ax.set_xlim([200.0, 50000.0])
ax.set_ylim([5, 15])

o_path = os.path.join(ThDSubduction_DIR, "r10_compare.pdf")
# fig.savefig(o_path)
# print("figure saved: ", o_path)

### 2d case summary

#### Generate the slab morphology analysis for a group of 2d case

In [ ]:
time_interval = 0.1e6

# read the list of cases
case_list, _, _, _ = ReadBasicInfoGroup(group_dir_2d)

# write a .sh file to run command in the system
py_temp_file = os.path.join(ASPECT_LAB_DIR, 'py_temp.sh')
py_commands = []

for _case in case_list:
  case_dir = os.path.join(group_dir_2d, _case)
  py_command = "python -m shilofue.TwoDSubduction0.VtkPp morph_case_parallel -i %s -ti %.2e\n" % (case_dir, time_interval)
  py_commands.append(py_command)

with open(py_temp_file, 'w') as fout:
  fout.write("#!/bin/bash\n")
  for py_command in py_commands:
    fout.write(py_command)

#### Update case summary and table

Note that the dip angle at 660 km needs to be entered by hand

In [ ]:
# for the project, no update, hand correct
write_for_paper = False; update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary_2d.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary_2d.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary_2d.tex')
# for one group, update from group
# write_for_paper = False; update_from_group = True; o_path = os.path.join(group_dir_2d, 'case_summary_2d.txt'); o_path_csv = os.path.join(group_dir_2d, 'case_summary_2d.csv'); o_path_tex = os.path.join(group_dir_2d, 'case_summary_2d.tex')

# for export to a latex table for the paper
# write_for_paper = True; update_from_group = False; o_path = os.path.join(ThDSubduction_DIR, 'case_summary_2d_3d_paper.txt'); o_path_csv = os.path.join(ThDSubduction_DIR, 'case_summary_2d.csv'); o_path_tex = os.path.join(ThDSubduction_DIR, 'case_summary_2d_3d_paper.tex')

all_actions=["ages", "include_Ps", "strength", "t660", "t800", "t1000", "dip660"]

img_dir = os.path.join(group_dir_2d, 'img')
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

Case_Summary_2d = CASE_SUMMARY_2D(VISIT_OPTIONS=VISIT_OPTIONS_2D)

# import old result if it exists 
if os.path.isfile(o_path_csv):
  Case_Summary_2d.import_file(o_path_csv)
  Case_Summary_2d.Update(actions=all_actions)

# import the new directory
if update_from_group:
  Case_Summary_2d.import_directory(group_dir_2d, actions=all_actions)

# output file output
if write_for_paper: 
  Case_Summary_2d.write_file_for_3d_paper(o_path_tex)
else:
  Case_Summary_2d.write_file(o_path)
  Case_Summary_2d.write_file(o_path_csv)
  Case_Summary_2d.write_file(o_path_tex)

#### Export the dip angle at 660 km

The following code block loop over all the twod cases and output the cases where a dip angle at 660 km could be pulled out

In [ ]:
yr = 365 * 24 * 3600.0
radius = 6371e3

outputs = ""
for i in range(Case_Summary_2d.n_case):
    local_dir_2d = Case_Summary_2d.ab_paths[i]
    case_name = Case_Summary_2d.cases[i]
    dip660 = np.nan
    try:
        dip660 = GetSlabDipAt660(local_dir_2d)
        Case_Summary_2d.update_dip660(i, dip660*180.0/np.pi)
    except Exception:
        pass
    outputs += "case_name: %s, dip660: %.4f (%.4f degree)\n" % (case_name, dip660, dip660/np.pi*180.0)

In [ ]:
print(outputs)

### Now output the file again

In [ ]:
# output file output
if write_for_paper: 
  Case_Summary_2d.write_file_for_3d_paper(o_path_tex)
else:
  Case_Summary_2d.write_file(o_path)
  Case_Summary_2d.write_file(o_path_csv)
  Case_Summary_2d.write_file(o_path_tex)

#### Compare to analytical solution from Capitanio 2013 paper

In [ ]:
IndexByValue = lambda array_1d, val: np.argmin(abs(array_1d - val))

# case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"
case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss1000000.0"
# case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss1000000.0"
# case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA40.0_OA20.0_width80_ss1000000.0"


slab_morph_file = os.path.join(case_dir_2d, 'vtk_outputs', "slab_morph_t1.00e+05.txt")

Visit_Options = VISIT_OPTIONS_2D(case_dir_2d)
Visit_Options.Interpret()
geometry = Visit_Options.options['GEOMETRY']
Ro = Visit_Options.options['OUTER_RADIUS']

SlabPlot2D = SLABPLOT2D('2d')
SlabPlot2D.ReadHeader(slab_morph_file)
SlabPlot2D.ReadData(slab_morph_file)
col_pvtu_time = SlabPlot2D.header['time']['col']
col_pvtu_trench = SlabPlot2D.header['trench']['col']
col_pvtu_slab_depth = SlabPlot2D.header['slab_depth']['col']
col_pvtu_sp_v = SlabPlot2D.header['subducting_plate_velocity']['col']
col_dip_100 = SlabPlot2D.header['100km_dip']['col']
times = SlabPlot2D.data[:, col_pvtu_time]
trenches = SlabPlot2D.data[:, col_pvtu_trench]
slab_depths = SlabPlot2D.data[:, col_pvtu_slab_depth]
sp_vs = SlabPlot2D.data[:, col_pvtu_sp_v]
time_interval = times[1] - times[0]
dip100s = SlabPlot2D.data[:, col_dip_100]

if geometry == "chunk":
    trenches_migration_length = (trenches - trenches[0]) * Ro  # length of migration
elif geometry == 'box':
    trenches_migration_length = trenches - trenches[0]
else:
    raise ValueError('Invalid geometry')
slab_depthes = SlabPlot2D.data[:, col_pvtu_slab_depth]
trench_velocities = np.gradient(trenches_migration_length, times)
sink_velocities = np.gradient(slab_depthes, times)

# time of slab tip reaching 660 km and the index in the list
sfunc = interp1d(slab_depths, times, assume_sorted=True)
t660_2d = sfunc(400e3)
i660_2d = IndexByValue(times, t660_2d)
Vsp660_2d = sp_vs[i660_2d]
dip100_660 = dip100s[i660_2d]
Vtrench660s = trench_velocities[i660_2d-2: i660_2d+3]

print(t660_2d)
print(i660_2d)
print(Vsp660_2d)
print(dip100_660)
print(Vtrench660s)


In [ ]:
def GetInfo(case_dir_2d, query_depth):
    '''
    Get infomation from one case
    '''
    slab_morph_file = os.path.join(case_dir_2d, 'vtk_outputs', "slab_morph_t1.00e+05.txt")

    Visit_Options = VISIT_OPTIONS_2D(case_dir_2d)
    Visit_Options.Interpret()
    geometry = Visit_Options.options['GEOMETRY']
    Ro = Visit_Options.options['OUTER_RADIUS']

    SlabPlot2D = SLABPLOT2D('2d')
    SlabPlot2D.ReadHeader(slab_morph_file)
    SlabPlot2D.ReadData(slab_morph_file)
    col_pvtu_time = SlabPlot2D.header['time']['col']
    col_pvtu_trench = SlabPlot2D.header['trench']['col']
    col_pvtu_slab_depth = SlabPlot2D.header['slab_depth']['col']
    col_pvtu_sp_v = SlabPlot2D.header['subducting_plate_velocity']['col']
    col_pvtu_op_v = SlabPlot2D.header['overiding_plate_velocity']['col']
    col_dip_100 = SlabPlot2D.header['100km_dip']['col']
    times = SlabPlot2D.data[:, col_pvtu_time]
    trenches = SlabPlot2D.data[:, col_pvtu_trench]
    slab_depths = SlabPlot2D.data[:, col_pvtu_slab_depth]
    sp_vs = SlabPlot2D.data[:, col_pvtu_sp_v]
    op_vs = SlabPlot2D.data[:, col_pvtu_op_v]
    time_interval = times[1] - times[0]
    dip100s = SlabPlot2D.data[:, col_dip_100]

    if geometry == "chunk":
        trenches_migration_length = (trenches - trenches[0]) * Ro  # length of migration
    elif geometry == 'box':
        trenches_migration_length = trenches - trenches[0]
    else:
        raise ValueError('Invalid geometry')
    slab_depthes = SlabPlot2D.data[:, col_pvtu_slab_depth]
    trench_velocities = np.gradient(trenches_migration_length, times)
    sink_velocities = np.gradient(slab_depthes, times)

    # time of slab tip reaching 660 km and the index in the list
    sfunc = interp1d(slab_depths, times, assume_sorted=True)
    t_2d = sfunc(query_depth)
    i_2d = IndexByValue(times, t_2d)
    Vsp_2d = sp_vs[i_2d]
    dip100 = dip100s[i_2d]
    Vtrench = -1.0 * op_vs[i_2d]
    return [Vsp_2d, dip100, Vtrench]

def GetAnalyticalResults(ts, dip, slab_depth, eta_M):

    l = 4000e3
    W = 180e3

    Cap13 = CAP13(eta_M=eta_M, delta_rho=70.0)
    results, coeffs = Cap13.CalculateVelocities(l, ts, dip, W, slab_depth)
    return results

In [ ]:
plt.style.use('publication_2d_morph_compare_sub2')

year = 365.0 * 24 * 3600.0
ma = 1e6 * year # convert ma to s
cm = 1e-2 # cm to m
cm_per_yr = cm / year # cm/yr to m/s

# no Peierls
# case_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22")
# case_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss1000000.0")
# case_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA40.0_OA20.0_width80_h1000_ss1000000.0")
# include Peierls
# case_dir_2d = os.path.join(TwoDSubduction_DIR, "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss1000000.0")
# case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss1000000.0"
case_dir_2d = "/mnt/lochy/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA40.0_OA20.0_width80_ss1000000.0"

query_depth = 660e3
ages = 40.0
ts = ages * ma
eta_M = 1e21

ModelResults = GetInfo(case_dir_2d, query_depth)
Vsp_2d, dip100, Vtrench = ModelResults[0], ModelResults[1], ModelResults[2]
print("ModelResults: ", ModelResults) # debug

AnalyticResults = GetAnalyticalResults(ts, dip100, query_depth, eta_M)
Vsp_Cap13, Vtrench_Cap13 = AnalyticResults['u'], AnalyticResults['u_T']

print(Vsp_2d/cm, Vtrench/cm)
print(Vsp_Cap13/cm_per_yr, Vtrench_Cap13/cm_per_yr)


import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
plt.style.use('publication_2d_morph_compare_sub2')


year = 365.0 * 24 * 3600.0
ma = 1e6 * year # convert ma to s
cm = 1e-2 # cm to m
cm_per_yr = cm / year # cm/yr to m/s

# model results
ModelResults = GetInfo(case_dir_2d, query_depth)
Vsp_2d, dip100, Vtrench = ModelResults[0], ModelResults[1], ModelResults[2]


trench_retreat = [Vtrench/cm]
plate_sliding = [Vsp_2d/cm]

# analytic model
l = 4000e3
W = 180e3


ts = np.linspace(0.0, 160.0, 100) * ma
dip = dip100

AnalyticResults = GetAnalyticalResults(ts, dip, query_depth, 1e21)
AnalyticResults1 = GetAnalyticalResults(ts, dip, query_depth, 5e20)
AnalyticResults2 = GetAnalyticalResults(ts, dip, query_depth, 5e21)
AnalyticResults3 = GetAnalyticalResults(ts, dip, query_depth, 2e21)



fig = plt.figure(tight_layout=True, figsize=(30, 12))
gs = gridspec.GridSpec(1, 2)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax1.plot(ts/ma, AnalyticResults['u_T']/cm_per_yr, 'k', label="net trench velocity, dip = %.2f" % (180.0 * dip / np.pi))
ax1.plot(ts/ma, AnalyticResults['u']/cm_per_yr, 'r', label="sliding velocity")
ax1.plot(ages, trench_retreat, 'ko')
ax1.plot(ages, plate_sliding, 'ro')
ax1.set_ylim((-20.0, 20.0))
ax1.set_xlabel('Age (ma)')
ax1.set_ylabel('Velocity (cm/yr)')
ax1.grid()
ax1.legend()

ax2.plot(ts/ma, AnalyticResults['u_T']/cm_per_yr, 'k', label="net trench velocity, 1e21 Pa*s")
ax2.plot(ts/ma, AnalyticResults['u']/cm_per_yr, 'r', label="sliding velocity")
ax2.plot(ts/ma, AnalyticResults1['u_T']/cm_per_yr, 'k--', label="5e20 Pa*s")
ax2.plot(ts/ma, AnalyticResults1['u']/cm_per_yr, 'r--')
ax2.plot(ts/ma, AnalyticResults2['u_T']/cm_per_yr, 'k', linestyle="dotted", label="5e21 Pa*s")
ax2.plot(ts/ma, AnalyticResults2['u']/cm_per_yr, 'r', linestyle="dotted")
ax2.plot(ts/ma, AnalyticResults3['u_T']/cm_per_yr, 'k', linestyle="dashdot", label="2e21 Pa*s")
ax2.plot(ts/ma, AnalyticResults3['u']/cm_per_yr, 'r', linestyle="dashdot")
ax2.plot(ages, trench_retreat, 'ko')
ax2.plot(ages, plate_sliding, 'ro')
ax2.set_ylim((-20.0, 20.0))
ax2.set_xlabel('Age (ma)')
ax2.grid()
ax2.legend()

# fileout = os.path.join(ThDSubduction_DIR, "img", "Capitanio13_compare_h660km_dip%.2f.pdf" % (dip[0] * 180.0 / np.pi))
fileout = os.path.join(case_dir_2d, "img", "Capitanio13_compare_h660km_dip%.2f.pdf" % (dip * 180.0 / np.pi))
fig.savefig(fileout)
print("File saved %s" % fileout)

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
from matplotlib import get_configdir
get_configdir()